<a href="https://colab.research.google.com/github/ErvinHalimsurya/bangkit-capstone-ocr/blob/main/OCR_text_detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from imutils.object_detection import non_max_suppression
from google.colab.patches import cv2_imshow
from imutils import paths
import tensorflow as tf
import numpy as np
import time
import cv2
import os

In [ ]:
!git clone https://github.com/dilhelh/opencv-text-detection.git

Cloning into 'opencv-text-detection'...
remote: Enumerating objects: 11, done.
remote: Total 11 (delta 0), reused 0 (delta 0), pack-reused 11
Unpacking objects: 100% (11/11), done.


In [ ]:
!python /content/opencv-text-detection/text_detection.py --image /content/opencv-text-detection/images/lebron_james.jpg \
	--east /content/opencv-text-detection/frozen_east_text_detection.pb

[INFO] loading EAST text detector...
[INFO] text detection took 0.561433 seconds
: cannot connect to X server 


In [ ]:
!wget -q https://github.com/sayakpaul/Adventures-in-TensorFlow-Lite/releases/download/v0.11.0/coco_text_100.tar.gz
!tar xf coco_text_100.tar.gz

In [ ]:
IMAGE_LIST = list(paths.list_images('/content/coco_text_100'))
IMG_SIZE = 320

In [ ]:
def representative_dataset_gen():
    for image_path in IMAGE_LIST:
        image = cv2.imread(image_path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = image.astype("float32")
        mean = np.array([123.68, 116.779, 103.939][::-1], dtype="float32")
        image -= mean
        image = np.expand_dims(image, axis=0)
        yield [image]

In [ ]:
quantization = "dr" #@param ["dr", "int8", "float16"]
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file='/content/opencv-text-detection/frozen_east_text_detection.pb', 
    input_arrays=['input_images'],
    output_arrays=['feature_fusion/Conv_7/Sigmoid', 'feature_fusion/concat_3'],
    input_shapes={'input_images': [1, 320, 320, 3]}
)

converter.optimizations = [tf.lite.Optimize.DEFAULT]

if quantization=="float16":
    converter.target_spec.supported_types = [tf.float16]
elif quantization=="int8":
    converter.representative_dataset = representative_dataset_gen
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8

tflite_model = converter.convert()
open('TextDetection_Model_{}.tflite'.format(quantization), 'wb').write(tflite_model)

48150544

In [ ]:
# define the two output layer names for the EAST detector model that
# we are interested -- the first is the output probabilities and the
# second can be used to derive the bounding box coordinates of text
layerNames = [
    "feature_fusion/Conv_7/Sigmoid",
    "feature_fusion/concat_3"]

In [ ]:
!ls -lh *.tflite
!ls -lh /content/opencv-text-detection/frozen_east_text_detection.pb

-rw-r--r-- 1 root root 24M Jun 12 14:10 TextDetection_Model_dr.tflite
-rw-r--r-- 1 root root 46M Jun 12 14:11 TextDetection_Model_float16.tflite
-rw-r--r-- 1 root root 24M Jun 12 14:10 TextDetection_Model_int8.tflite
-rw-r--r-- 1 root root 93M Jun 12 14:09 /content/opencv-text-detection/frozen_east_text_detection.pb
